In [35]:
import numpy as np
import random
import json
from google.colab import drive
import keras
!pip install -q google-generativeai
import os
import google.generativeai as genai
import base64
from google.generativeai import protos
from IPython.display import Audio
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio
from google.colab import userdata

In [38]:
drive.mount('/content/drive')
GOOGLE_API_KEY = userdata.get('GOOGLE2_API')
genai.configure(api_key=GOOGLE_API_KEY)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
with open("/content/drive/MyDrive/rapbot_text_all.txt", "r", encoding="utf-8") as f:
    text_All = f.read()

meta = json.load(open("/content/drive/MyDrive/rapbot_char_meta.json", "r"))

chars = meta["chars"]
char_indices = meta["char_indices"]
indices_char = {int(k): v for k, v in meta["indices_char"].items()}

print("Unikalūs simboliai:", len(chars))


Unikalūs simboliai: 40


In [40]:
maxlen = 80

model_path = "/content/drive/MyDrive/rap_lyrics_lstm_v3.keras"
model = keras.models.load_model(model_path)

print("Modelis sėkmingai užkrautas!")
model.summary()


Modelis sėkmingai užkrautas!


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_13 (LSTM)                  │ (None, 128)            │        86,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 40)             │         5,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 183,378 (716.32 KB)

 Trainable params: 91,688 (358.16 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 91,690 (358.17 KB)

In [41]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds + 1e-7) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


In [42]:
def generate_text(seed, length=400, temperature=0.7):
    if len(seed) < maxlen:
        seed = seed.rjust(maxlen)

    generated = seed[:maxlen]
    text = generated

    print("------ Seed ----")
    print(seed)
    print("------ Generuojama ----")

    for i in range(length):
        x_pred = np.zeros((1, maxlen, len(chars)))

        for t, char in enumerate(generated):
            if char in char_indices:
                x_pred[0, t, char_indices[char]] = 1.0

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = indices_char[next_index]

        text += next_char
        generated = generated[1:] + next_char

    return text


In [43]:
model_google = genai.GenerativeModel('gemini-2.5-flash')

In [44]:
def grammar_fix(text: str, temperature: float = 0.3) -> str:
    prompt = (
        "You are a grammar fixer. "
        "Take the following rap lyrics and correct grammar, punctuation and spelling mistakes, "
        "but keep the original style and vibe, "
        "you can try to manipulate the last words of the lines to make the text rhyme,"
        "but keep other, not corrected, words intact,"
        "print just the corrected version out, do not generate the '*' symbol"
        f"Original:\n{text}\n\nCorrected:\n"
    )
    response = model_google.generate_content(prompt)
    return response.text.strip()

In [ ]:
def speak(text, lang="en"):
    tts = gTTS(text=text, lang=lang)
    tts.save("output.mp3")
    return Audio("output.mp3", autoplay=True)

In [28]:
with open("/content/drive/MyDrive/shared_seed.txt", "r", encoding="utf-8") as f:
    seed_text = f.read().strip()
generated = generate_text(seed_text, length=150, temperature=0.8)
last_line = generated.strip().split("\n")[-1]
with open("/content/drive/MyDrive/shared_seed.txt", "w", encoding="utf-8") as f:
    f.write(last_line)
print("\n\n==== Sugeneruotas tekstas ====\n")
print(generated)


------ Seed ----
                                                                           the b
------ Generuojama ----


==== Sugeneruotas tekstas ====

                                                                           the blood
whats she fuck it say in this on parted of the dick to neid is somemy shought me
when its rock that i stard what we and you i be girl
you can you


In [45]:
fixed = grammar_fix(generated)
print(fixed)

The blood
What's she fuckin' say in this? On part of the dick,
The need is somethin' she brought, girl.
When it's rock that I start, what we and you, I'll be, girl.
You can, you can.


In [46]:
speak(fixed)